## This notebook generates matrix table for the 30x validation survindel2 table

In [1]:
%%configure -f
{"driverMemory": "6000M"}

In [2]:
import hail as hl
hl.init(sc)

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.1.2-amzn-0
SparkUI available at
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.80-4ccfae1ff293
LOGGING: writing to 

In [3]:
## list all resources  used in this notebook 
survindel2_uri="DUP.30x.for_hail.vcf.gz"
whiltelist_region_bed_uri = "resources_broad_hg38_v0_wgs_calling_regions.hg38.merged.autosome_only-minus_excl_regions.bed"
validation30x_sample_txt_uri = "SG10K-SV-Release-1.3_30xValidation.samples.txt"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
mt = hl.import_vcf(survindel2_uri, reference_genome="GRCh38", force_bgz=True)
mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        SVTYPE: str, 
        END: int32, 
        N_SAMPLES: int32, 
        N_SVS: int32, 
        GCF: float64, 
        SVSIZE: int32
    }
----------------------------------------
Entry fields:
    'GT': call
    'FT': str
    'AR': int32
    'SR': int32
    'RR': int32
    'RS': int32
    'RE': int32
    'CN': int32
    'IL': int32
    'WR': int32
    'DFL': int32
    'DDL': int32
    'DDR': int32
    'DFR': int32
    'CID': int32
    'OW': int32
    'DHFC': float64
    'DHBFC': float64
    'DHFFC': float64
    'DHSP': int32
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------

In [5]:
## Filter out relevant samples
sample_ht = hl.import_table(validation30x_sample_txt_uri).key_by('s')

print(sample_ht.count()) ## 5487
print(mt.count()) ## 

mt = mt.filter_cols(hl.is_defined(sample_ht[mt.col_key]))

print(mt.count()) ## 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1922
(43622, 1922)
(43622, 1922)
2024-05-06 08:56:39 Hail: INFO: Reading table without type imputation
  Loading field 's' as type str (not specified)
2024-05-06 08:56:54 Hail: INFO: Coerced sorted dataset
2024-05-06 08:57:13 Hail: INFO: Coerced sorted dataset

In [6]:
# Check if the genotype PASS the QC
mt = mt.annotate_entries(
    FS = hl.case()
            .when((mt.GT.is_hom_ref()) & (mt.FT == "PASS"), "PASS")
            .when((mt.GT.is_het()) &(mt.FT == "PASS") & (mt.DHBFC > 1.3), "PASS")
            .when((mt.GT.is_hom_var()) &(mt.FT == "PASS") & (mt.DHBFC > 1.3), "PASS")
            .default("FAIL")            
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Check if the site PASS across all samples
# Variant site is marked as FAIL if all samples have FS == FAIL

mt = mt.annotate_rows(
    site_is_pass = hl.agg.any(mt.FS == "PASS")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Define filters
filters = [
    {'key': 'PASS', 'expr': ( mt.rows().select("site_is_pass") == True ) },
    {'key': 'FAIL', 'expr': ( mt.rows().select("site_is_pass") == False )  },
]

ht = mt.rows()

# Compute filters
ht_filters = ht.annotate(
    filters=hl.set(hl.filter(
        lambda x: hl.is_defined(x),
        [hl.or_missing(
            d['expr'],
            d['key']
        ) for d in filters]
    ))
)

# Copy filters into main sample ht
mt= mt.annotate_rows(
    filters = mt.filters.union(ht_filters[mt.row_key].filters)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
##  because we want to only carry fwd entries wit at least one hom-ref and drop monomorphic entries
##   and there is a need to put all genotypes that fail FORMAT/FT to fORMAT/GT = `./.`


mt = mt.annotate_entries(
    GT = hl.case()
            .when((mt.GT.is_hom_ref()) & (mt.FT == "PASS"), mt.GT)
            .when((mt.GT.is_het()) &(mt.FT == "PASS") & (mt.DHBFC > 1.3), mt.GT)
            .when((mt.GT.is_hom_var()) &(mt.FT == "PASS") & (mt.DHBFC > 1.3),mt.GT)
            .default( hl.null(hl.tcall) )            
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
## filter relevant variant
## 2- contains "PASS", "{fail}" FILTER entries We only carry fwd "FILTER=PASS" entries 
mt = mt.filter_rows(mt.filters.length() == 0, keep=True)
print(mt.count()) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(43622, 1922)
2024-05-06 08:58:13 Hail: INFO: Coerced sorted dataset
2024-05-06 08:58:19 Hail: INFO: Coerced sorted dataset

In [11]:
# Remove monomorphic variants

# run variant qc
mt = hl.variant_qc(mt)

# annotate the matrix table
mt = mt.annotate_rows(
    n_called = mt.variant_qc.n_called,
    n_hom_ref = hl.agg.count_where(mt.GT.is_hom_ref()),
    n_het = hl.agg.count_where(mt.GT.is_het()),
    n_hom_alt = hl.agg.count_where(mt.GT.is_hom_var())
)

mt = mt.annotate_rows(
   n_non_ref = mt.n_hom_alt + mt.n_het 
)

# Remove monomorphic variants
mt = mt.filter_rows(
    (mt.n_called == mt.n_hom_ref) | 
    (mt.n_called == mt.n_het) | 
    (mt.n_called == mt.n_hom_alt),
    keep = False
)
print("Samples: %d; Variants: %d; Entries: %d" % (mt.count_cols(), mt.count_rows(), mt.entries().count()))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Samples: 1922; Variants: 22625; Entries: 43485250
2024-05-06 08:58:38 Hail: INFO: Coerced sorted dataset
2024-05-06 08:58:44 Hail: INFO: Coerced sorted dataset
2024-05-06 08:58:50 Hail: INFO: Coerced sorted dataset
2024-05-06 08:59:09 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'
2024-05-06 08:59:15 Hail: INFO: Coerced sorted dataset
2024-05-06 08:59:21 Hail: INFO: Coerced sorted dataset

In [12]:
# FILTER: sites with no hom_ref call
mt = mt.filter_rows(mt.n_called == mt.n_non_ref, keep = False)
print("Samples: %d; Variants: %d; Entries: %d" % (mt.count_cols(), mt.count_rows(), mt.entries().count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Samples: 1922; Variants: 22554; Entries: 43348788
2024-05-06 08:59:45 Hail: INFO: Coerced sorted dataset
2024-05-06 08:59:51 Hail: INFO: Coerced sorted dataset
2024-05-06 08:59:57 Hail: INFO: Coerced sorted dataset
2024-05-06 09:00:22 Hail: INFO: Coerced sorted dataset
2024-05-06 09:00:28 Hail: INFO: Coerced sorted dataset

In [13]:
print(mt.count()) ## 


## load the predefine whitlist region (ie not low-cpmplexity, telemore, centromere, ...) 
whitelist_region= hl.import_bed(whiltelist_region_bed_uri, reference_genome='GRCh38')

## filter relevant variant
mt = mt.filter_rows(
    True
    & ((~hl.is_defined(mt.info.SVSIZE))                         ## We only carry fwd entries with INFO/SVSIZE undefined 
       | (mt.info.SVSIZE >= 50)                                 ##                            or  INFO/SVSIZE > 50bp
       | (mt.info.SVSIZE <= 10000000))                          ##                            or INFO/SVSIZE < 10,000,000bp 
    & (hl.is_defined(whitelist_region[mt.locus]))               ## We only carry fwd  whitelist region contained SV 
    , keep=True)

print(mt.count()) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22554, 1922)
(21377, 1922)
2024-05-06 09:00:54 Hail: INFO: Coerced sorted dataset
2024-05-06 09:01:00 Hail: INFO: Coerced sorted dataset
2024-05-06 09:01:18 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2024-05-06 09:01:25 Hail: INFO: Coerced sorted dataset
2024-05-06 09:01:30 Hail: INFO: Coerced sorted dataset
2024-05-06 09:01:31 Hail: INFO: Coerced sorted dataset

In [14]:
release14_surv_mt_uri = "SG10K_SV_SURVINDEL2.n1922.m21377.30xvalidation.mt"
mt.write(release14_surv_mt_uri, overwrite=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-06 09:02:22 Hail: INFO: Coerced sorted dataset
2024-05-06 09:02:28 Hail: INFO: Coerced sorted dataset
2024-05-06 09:02:28 Hail: INFO: Coerced sorted dataset
2024-05-06 09:03:29 Hail: INFO: wrote matrix table with 21377 rows and 1922 columns in 9 partitions to SG10K_SV_SURVINDEL2.n1922.m21377.30xvalidation.mt
    Total size: 816.30 MiB
    * Rows/entries: 816.29 MiB
    * Columns: 6.94 KiB
    * Globals: 11.00 B
    * Smallest partition: 1382 rows (54.52 MiB)
    * Largest partition:  2594 rows (99.13 MiB)